In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [39]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("your_api_key")

In [40]:
aws_access_key_id = "you_access_key"
aws_secret_acess_key = "your_secret_access_key"
region_name = "ap-south-1"

In [5]:
import boto3
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [7]:
bedrock = boto3.client(service_name = "bedrock-runtime" ,
                       region_name = region_name,
                       aws_access_key_id = aws_access_key_id,
                       aws_secret_access_key = aws_secret_acess_key
                       )

In [24]:
prompt_template = """ 
Human: Use the following pieces of context to provides a concise answer to the question at the end but use atleast summerize with 
250 words with detailed explaninations. If you don't know the answer, just say that you don't know, dont't try to generte any answer just say
i am not able to answer these question and give I dont't know

<context>
{context}
</context

Question: {question}
Assistant
"""

In [25]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [26]:
loader = PyPDFDirectoryLoader("../data")
documents = loader.load()

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

In [28]:
docs = text_splitter.split_documents(documents)

In [29]:
docs[0]

Document(metadata={'source': '..\\data\\2007.03639v3.pdf', 'page': 0, 'page_label': '1'}, page_content='1\nHuman Trajectory Forecasting in Crowds:\nA Deep Learning Perspective\nParth Kothari, Sven Kreiss, and Alexandre Alahi\nAbstract—Since the past few decades, human trajectory fore-\ncasting has been a ﬁeld of active research owing to its nu-\nmerous real-world applications: evacuation situation analysis,\ndeployment of intelligent transport systems, trafﬁc operations, to\nname a few. Early works handcrafted this representation based\non domain knowledge. However, social interactions in crowded\nenvironments are not only diverse but often subtle. Recently,\ndeep learning methods have outperformed their handcrafted\ncounterparts, as they learned about human-human interactions\nin a more generic data-driven fashion. In this work, we present\nan in-depth analysis of existing deep learning-based methods\nfor modelling social interactions. We propose two knowledge-\nbased data-driven meth

In [30]:
vectorstore_faiss = FAISS.from_documents(
    docs ,
    embeddings
)

vectorstore_faiss.save_local("faiss_index")

In [31]:
llm = Bedrock(
    model_id = "meta.llama3-8b-instruct-v1:0",
    client = bedrock,
    model_kwargs = {'max_gen_len':512}
)

In [32]:
prommpt = PromptTemplate(
    template=prompt_template,
    input_variables=['context' , 'question']
)

In [36]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore_faiss.as_retriever(
        search_type = "similarity" ,
        search_kwargs = {"k":1}
    ),
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prommpt}
)

In [37]:
query = "what is trajnet++ forcasting model"
answer = qa({"query":query})
answer['result']

"Based on the provided context, TrajNet++ is a human trajectory forecasting model that is interaction-centric. This means that it focuses on scenes where interactions take place, which is achieved through a defined trajectory categorization process. The model is designed to provide an extensive evaluation system that includes novel collision-based metrics to measure the physical feasibility of its predictions.\n\nIn simpler terms, TrajNet++ is a model that predicts the future trajectory of humans based on their interactions with each other and their environment. The model is unique in that it focuses on scenes where interactions occur, such as pedestrian collisions or social gatherings, and uses novel metrics to evaluate the physical feasibility of its predictions.\n\nThe model's design is based on an in-depth analysis of existing interaction module designs in the literature, and it proposes two domain-knowledge based interaction models. The TrajNet++ model is evaluated using a compreh

In [38]:
query = "who is elon musk"
answer = qa({"query":query})
answer['result']

'I am not able to answer these questions. The provided context is related to trajectory forecasting models, and it does not mention Elon Musk. Therefore, I am unable to provide an answer to the question about who Elon Musk is. If you have any other questions or context related to the topic of trajectory forecasting models, I would be happy to help.'